In [70]:
import pandas as pd
import numpy as np
import mysql.connector

connect = mysql.connector.connect(host='localhost',database='temp',user='root',password='pw')
cursor = connect.cursor()

Q1: Consider only the rows with country_id = "BDV" (there are 844 such rows). For each site_id, we can compute the number of unique user_id's found in these 844 rows. Which site_id has the largest number of unique users? And what's the number?

In [71]:
query = '''
    select site_id, count(distinct(user_id)) as cnt 
    from q11 
    where country_id ="BDV" 
    group by site_id 
    order by cnt desc
;
'''
cursor.execute(query)
cursor.fetchall()

[('5NPAU', 544), ('N0OTG', 90), ('3POLC', 2)]

Q2: Between 2019-02-03 00:00:00 and 2019-02-04 23:59:59, there are four users who visited a certain site more than 10 times. Find these four users & which sites they (each) visited more than 10 times. (Simply provides four triples in the form (user_id, site_id, number of visits) in the box below.)

In [72]:
query2 = '''
    select * from 
        (select user_id, site_id, count(ts) as num_visit 
        from q11 
        where ts between '2019-02-03 00:00:00' and '2019-02-04 23:59:59'
        group by user_id, site_id) t 
    where num_visit > 10
; 
'''
cursor.execute(query2)
cursor.fetchall()

[('LC3C7E', '3POLC', 15),
 ('LC3A59', 'N0OTG', 26),
 ('LC06C3', 'N0OTG', 25),
 ('LC3C9D', 'N0OTG', 17)]

Q3: For each site, compute the unique number of users whose last visit (found in the original data set) was to that site. For instance, user "LC3561"'s last visit is to "N0OTG" based on timestamp data. Based on this measure, what are top three sites? (hint: site "3POLC" is ranked at 5th with 28 users whose last visit in the data set was to 3POLC; simply provide three pairs in the form (site_id, number of users).)

In [73]:
query3 = '''
select site_id, count(distinct(user_id)) as l_user_cnt
from 
    (select user_id, site_id, row_number() over (partition by user_id order by ts desc) as rn 
    from q11) t 
where rn = 1 
group by site_id
order by l_user_cnt desc
; 

'''
cursor.execute(query3)
cursor.fetchall()

[('5NPAU', 992),
 ('N0OTG', 561),
 ('QGO3G', 289),
 ('GVOFK', 42),
 ('3POLC', 28),
 ('RT9Z6', 2),
 ('EUZ/Q', 1),
 ('JSUUP', 1)]

Q4: For each user, determine the first site he/she visited and the last site he/she visited based on the timestamp data. Compute the number of users whose first/last visits are to the same website. What is the number?

In [74]:
query4 = '''
select count(distinct(t1.user_id))
from 
    (select user_id, site_id as site_id_f, row_number() over (partition by user_id order by ts asc) as rn_first
    from q11) t1
inner join 
    (select user_id, site_id as site_id_l, row_number() over (partition by user_id order by ts desc) as rn_last
    from q11) t2 
on t1.user_id = t2.user_id
where t1.rn_first = 1 
and t2.rn_last = 1
and t1.site_id_f = t2.site_id_l 
; 
'''
cursor.execute(query4)
cursor.fetchall()
# The number of users whose first/last visits are to the same website: 1670 users

[(1670,)]

In [75]:
query4_2 = '''
select A.user_id, A.site_id_f, B.cnt_ts from 
   (select t1.user_id, site_id_f
        from 
            (select user_id, site_id as site_id_f, row_number() over (partition by user_id order by ts asc) as rn_first
            from q11) t1
        inner join 
            (select user_id, site_id as site_id_l, row_number() over (partition by user_id order by ts desc) as rn_last
            from q11) t2 
        on t1.user_id = t2.user_id
        where t1.rn_first = 1 
        and t2.rn_last = 1
        and t1.site_id_f = t2.site_id_l ) A
left join 
    (select user_id, count(distinct(ts)) as cnt_ts from q11 group by user_id) B
on A.user_id = B.user_id   
where  
B.cnt_ts >1  

; 
'''
cursor.execute(query4_2)
l = cursor.fetchall()
len(l) 

# if we exclude the users whose total number of visits is only one 
# The number of users whose first/last visits are to the same website: 409 users

409